In [2]:
# Stdlib
import sys
import os
import random

# Scientific computing
import numpy as np

# Network science
import networkx as nx

# Data handling and vis
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline


# Project source
sys.path.append("../../src/")
from synthetic import *
from utils import *
import copy

In [28]:
G=read_file('/Users/siddharthpatwardhan/Downloads/Archived/MRP_Figures/Figure2/multiplex_network-london.edgelist')

In [29]:
G, H, R_G, R_H, testset = format_system({'graphs':G,'pfi':0.5,'left':1,'right':2,'largest_component':False})

In [ ]:
X1,X2=n2v(R_G),n2v(R_H)

In [ ]:

for e in testset:
    print(np.linalg.norm(X1[e[0]]-X1[e[1]]),np.linalg.norm(X2[e[0]]-X2[e[1]]))
    print(testset[e])

In [ ]:
testset

In [14]:
def format_system(params):
    # Process parameters
    pfi = params["pfi"]
    graphs = params["graphs"]
    left = params["left"]
    right = params["right"]

    # Form "raw" duplex
    multiplex = graphs

    # Split into layers
    G, H = duplex_network(multiplex, left, right)

    # Observe partial information
    R_G, R_H, testset = partial_information(G, H, pfi)

    # Restrict to largest connected component (if specified)
    if params["largest_component"]:
        R_G_ = nx.Graph()
        R_H_ = nx.Graph()
        R_G_.add_nodes_from(R_G.nodes())
        R_H_.add_nodes_from(R_H.nodes())

        maxcc_R_G = max(nx.connected_components(R_G), key=len)
        maxcc_R_H = max(nx.connected_components(R_H), key=len)

        edges_R_G_ = set(R_G.subgraph(maxcc_R_G).edges())
        edges_R_H_ = set(R_H.subgraph(maxcc_R_H).edges())
        R_G_.add_edges_from(edges_R_G_)
        R_H_.add_edges_from(edges_R_H_)

        testset = {
            edge: gt_
            for edge, gt_ in testset.items()
            if edge in edges_R_H_ | edges_R_H_
        }

    return G, H, R_G, R_H, testset


def reconstruct_system(testset, G, H, G_, H_, params):
    cls = []
    scores = []
    gt = []

    _dot = lambda x, y: np.exp(-1*np.dot(np.transpose(x), y))

    for edge, gt_ in testset.items():
        i, j = edge
        gt.append(gt_)

        v_G_i = G_[i]
        v_G_j = G_[j]
        v_H_i = H_[i]
        v_H_j = H_[j]

        if params["metric"] == "logistic":
            d_G = 1 / (1 + _dot(v_G_i, v_G_j))
            d_H = 1 / (1 + _dot(v_H_i, v_H_j))
        else:
            d_G = np.linalg.norm(v_G_i - v_G_j)
            d_H = np.linalg.norm(v_H_i - v_H_j)

            if params["metric"] == "inverse":
                d_G = 1 / (d_G + 1e-16)
                d_H = 1 / (d_H + 1e-16)
            elif params["metric"] == "negexp":
                d_G = np.exp(-d_G)
                d_H = np.exp(-d_H)

        t_G = d_G / (d_G + d_H)
        t_H = 1 - t_G

        scores.append(t_G)

        cls_ = np.random.randint(2)
        if t_G != t_H:
            if np.random.rand() <= t_G:
                cls_ = 1
            else:
                cls_ = 0
        cls.append(cls_)

    return cls, scores, gt

def measure_performance(cls, scores, gt):
    # print(f"Classifications: {cls}\n Scores: {scores}\n GT: {gt}")
    acc = accuracy(gt, cls)
    auc = auroc(gt, scores)

    return acc, auc

In [21]:
G, H, R_G, R_H, testset=format_system({'pfi':0.5,'graphs':G,'left':1,'right':2,'largest_component':False})

In [36]:
G_=n2v(R_G)
H_=n2v(R_H)

In [ ]:
reconstruct_system(testset, G, H, G_, H_, {'metric':'logistic'})

In [10]:
random.choice(list(g.nodes()))

78

In [9]:
def overlap_multiplex(n=100,o=0.5):
    '''
    Input: n: Size of active nodes on each layer, overlapping nodes: Fraction of nodes appearing on both layers
    Output: [nx object layer 1, nx object layer 2]
    '''

    #create layer 1
    g1=nx.barabasi_albert_graph(n,1)
    g1.add_edges_from([(random.choice(list(g1.nodes())),random.choice(list(g1.nodes()))) for _ in range(10)])
    g1.remove_edges_from(nx.selfloop_edges(g1))

    #create layer 2
    g2=nx.barabasi_albert_graph(n,1)
    g2.add_edges_from([(random.choice(list(g2.nodes())),random.choice(list(g2.nodes()))) for _ in range(10)])
    g2.remove_edges_from(nx.selfloop_edges(g2))

    #relabel nodes
    shuffle=random.sample(g2.nodes(),n)
    mapping={i:shuffle[i] for i in g2.nodes()}
    g2=nx.relabel_nodes(g2,mapping)

    g1.add_nodes_from(list(range(n,2*n)))
    g2.add_nodes_from(list(range(n,2*n)))

    #overlapping nodes
    move=int((1-o)*n)
    mapping={i:(i+move)%(2*n) for i in g2.nodes()}
    g2=nx.relabel_nodes(g2,mapping)

    return [g1,g2]

In [1]:
G=overlap_multiplex()


NameError: name 'overlap_multiplex' is not defined

In [ ]:
G[1].edges()